In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Set up Selenium options and driver
options = Options()
options.add_argument("--disable-gpu")
# options.add_argument("--headless")  # Optional: Run in headless mode

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Navigate to the page
base_url = "https://www.nykaa.com/brands/loreal-paris/c/595?"
params = "ptype=brand&id=595&root=brand_menu,top_brands,L%27Oreal%20Paris"
paginationNumber = 1

# Allow initial time for the page to load
time.sleep(5)

# Scroll incrementally until all products are loaded
scroll_pause_time = 2  # Wait time after each scroll

# Lists to store data
productLinks = []
products = []

while True:

    if paginationNumber == 1:
        url = f"{base_url}{params}"
    else:
        url = f"{base_url}page_no={paginationNumber}&sort=popularity&ptype=brand&id=595&root=brand_menu,top_brands,L%27Oreal%20Paris&eq=desktop"
    
    # Load the Page
    driver.get(url)
    print(f"Scraping Page {paginationNumber}: {url}")
    time.sleep(3)
    
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Wait until the products are loaded
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.productWrapper.css-17nge1h'))
        )
    except TimeoutException:
        print(f"Scraped the Total number of pages: {paginationNumber - 1}")
        break

    # Parse the page content with BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    
    # Find all product items
    deygaOrganicProducts = soup.findAll('div', class_='productWrapper css-17nge1h')
    if not deygaOrganicProducts:
        print("No Products found on this page.")
        break

    # Extract product links
    for product in deygaOrganicProducts:
        # Extract the Stock Status
        stockTag = product.find('span', class_ = "css-lg5xc9")
        if stockTag:
            stock = stockTag.get_text(strip = True)
        else:
            stock = "In Stock"

        # Extract the Product Highlight
        highlightTag = product.find("li", class_ = "custom-tag css-1bse542")
        if highlightTag:
            highlight = highlightTag.get_text(strip = True)
        else:
            highlight = "None"

        # Extract product link
        linkTag = product.find('a', href=True)
        if linkTag:
            productLink = linkTag['href']
            # Check if the link is a relative URL and add the base URL if necessary
            if not productLink.startswith("http"):  # Relative URL
                productLink = "https://www.nykaa.com" + productLink
            productLinks.append({"link" : productLink, "stock" : stock, "highlight" : highlight})
                
    for productData in productLinks:

        link = productData["link"]
        stock = productData["stock"]
        highlight = productData["highlight"]
        
        driver.get(link)
        time.sleep(3)  # Allow time for the page to load

        # Parse individual product page
        productPageContent = driver.page_source
        productSoup = BeautifulSoup(productPageContent, 'html.parser')
    
        # Extract Product Name
        productTag = productSoup.find('h1', class_='css-1gc4x7i')
        if productTag:
            prodName = productTag.get_text(strip=True)
        else:
            prodName = "No Description Available"
    
        # Extract Ratings
        ratingTag = productSoup.find('div', class_="css-1m0y15j")
        if ratingTag:
            ratingText = ratingTag.find('div', class_="css-m6n3ou")
            if ratingText:
                ratings = ratingText.get_text(strip = True)
            else:
                ratings = "N/A"
        else:
            ratings = "N/A"

        # Extract the Ratings & Reviews Count
        ratingCountTag = productSoup.find('div', class_ = "css-1eip5u4")
        if ratingCountTag:
            ratingCount = ratingCountTag.get_text(strip = True)
        else:
            ratingCount = "None"

        # Extract the Offer Price and Original Price
        productDiv = productSoup.find('div', class_ = "css-1d0jf8e")

        # Extract the first two (span) tags
        if productDiv:
            spanTags = productDiv.find_all('span')
            # print(len(spanTags))

            originalPrice = "N/A"
            offerPrice = "N/A"

            if len(spanTags) == 2:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = "N/A"
            elif len(spanTags) == 4:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = spanTags[2].get_text(strip = True)

            # Print the Values
            # print(f"Original Price: {originalPrice}, Offer Price: {offerPrice}")

        else:
            print("Product div not found, defaulting to Unavailable for prices")

        # Extract the Discount
        discountTag = productSoup.find('span', class_ = "css-bhhehx")
        if discountTag:
            discount = discountTag.get_text(strip = True)
        else:
            discount = "No Discount"

        products.append({
            "Product Brand" : "L'Oreal Paris",
            "Product Name" : prodName,
            "Product Rating" : ratings,
            "Product Rating & Review Count" : ratingCount,
            "Product Original Price" : originalPrice, 
            "Product Offer Price" : offerPrice, 
            "Product Discount" : discount,
            "Product Highlight" : highlight,
            "Product Stock Status" : stock
        })

    # Clear the product links list to avoid the duplication
    productLinks.clear()

    paginationNumber += 1 # Increment the Page Number
    time.sleep(5)

# Close the browser after scraping
driver.quit()

Scraping Page 1: https://www.nykaa.com/brands/loreal-paris/c/595?ptype=brand&id=595&root=brand_menu,top_brands,L%27Oreal%20Paris
Scraping Page 2: https://www.nykaa.com/brands/loreal-paris/c/595?page_no=2&sort=popularity&ptype=brand&id=595&root=brand_menu,top_brands,L%27Oreal%20Paris&eq=desktop
Scraping Page 3: https://www.nykaa.com/brands/loreal-paris/c/595?page_no=3&sort=popularity&ptype=brand&id=595&root=brand_menu,top_brands,L%27Oreal%20Paris&eq=desktop
Scraping Page 4: https://www.nykaa.com/brands/loreal-paris/c/595?page_no=4&sort=popularity&ptype=brand&id=595&root=brand_menu,top_brands,L%27Oreal%20Paris&eq=desktop
Scraping Page 5: https://www.nykaa.com/brands/loreal-paris/c/595?page_no=5&sort=popularity&ptype=brand&id=595&root=brand_menu,top_brands,L%27Oreal%20Paris&eq=desktop
Scraping Page 6: https://www.nykaa.com/brands/loreal-paris/c/595?page_no=6&sort=popularity&ptype=brand&id=595&root=brand_menu,top_brands,L%27Oreal%20Paris&eq=desktop
Scraping Page 7: https://www.nykaa.com/br

In [6]:
# print(f"Products: \n {products}")

In [5]:
lorealParis = pd.DataFrame(products)
lorealParis

,Product Brand,Product Name,Product Rating,Product Rating & Review Count,Product Original Price,Product Offer Price,Product Discount,Product Highlight,Product Stock Status
0,L'Oreal Paris,L'Oreal Paris Voluminous Panorama Waterproof M...,4.4/5,1551ratings&281reviews,₹999,₹849,15% Off,BESTSELLER,In Stock
1,L'Oreal Paris,L'Oreal Paris Color Riche Satin Lipstick - 124...,4.3/5,129ratings&46reviews,₹750,₹713,5% Off,None,In Stock
2,L'Oreal Paris,L'Oreal Paris Infallible Matte Resistance Liqu...,4.3/5,6417ratings&739reviews,₹999,₹899,10% Off,BESTSELLER,In Stock
3,L'Oreal Paris,L'Oreal Paris Voluminous Lash Paradise Mascara...,4.4/5,18731ratings&2992reviews,₹899,₹764,15% Off,None,In Stock
4,L'Oreal Paris,"L'Oreal Paris Extraordinary Oil Hair Serum, An...",4.5/5,80981ratings&7073reviews,₹729,₹693,5% Off,BESTSELLER,In Stock
...,...,...,...,...,...,...,...,...,...
128,L'Oreal Paris,L'Oreal Paris Flash Cat Eye Eyeliner - Black(0...,4.3/5,979ratings&145reviews,₹799,N/A,No Discount,None,Out Of Stock
129,L'Oreal Paris,L'Oreal Paris Skin Perfect Age 40+ Anti-Aging ...,4.3/5,2438ratings&233reviews,₹649,N/A,No Discount,None,Out Of Stock
130,L'Oreal Paris,L'Oreal Paris Superliner Black Lacquer(1ml),4.1/5,870ratings&132reviews,₹799,N/A,No Discount,None,Out Of Stock
131,L'Oreal Paris,L'Oreal Paris Revitalift Triple Action Night C...,4.3/5,9ratings,₹1400,N/A,No Discount,None,Out Of Stock


In [7]:
# lorealParis.to_csv("C:/Users/Logiya Vidhyapathy/Documents/KGISL Data Science/Project Files and Documents/Capstone Project/L'Oreal Paris Products_Nykaa.csv", index = False)